In [3]:

import os
import pandas as pd
import re
from lxml import etree
import datetime
from datetime import datetime
import time

ns = {'tei': 'http://www.tei-c.org/ns/1.0', "xml": "http://www.w3.org/XML/1998/namespace"}




def readlist(pathtofile, filename):
      tree = etree.parse(pathtofile+"/"+filename)
      root = tree.getroot()
      return root

def readfiles(normrolelist, personlistxml, pathtofiles, type):
    Frame = pd.DataFrame()
    texts =[]
    norms = []
    persons = []
    sex = []
    relatedPersons =[]
    relatedSex = []
    regests = []
    regest_dates = []
    fromdate=[]
    todate=[]
    years = []
    listoferrors = []
    for filename in os.listdir(pathtofiles):
        print("--------------------------")
        print(filename)
        regesttree = etree.parse(pathtofiles+"/"+filename)
        regestroot = regesttree.getroot()
        actualregest = filename
        creationdate = regestroot.find(".//tei:creation/tei:date", ns)
        origindate = regestroot.find(".//tei:history/tei:origin", ns)
        if creationdate is not None:
           date = createdate(creationdate)
        elif origindate is not None:
           date = createdate(origindate)
        else:
           date = " "
        Nodes = regestroot.findall(".//tei:roleName[@type='"+type+"']", ns)
        for rs in Nodes:
            try:
                print("---------")
                print("-Node: %s" %rs.tag)
                print("--Attribute: %s" %rs.attrib)
                text = rs.xpath("normalize-space(.)")
                print("--Text: %s" %text)
                normed = normrole(normrolelist, text)

                if 'corresp' in rs.attrib:
                    relatedPerson = rs.attrib['corresp'].replace("#", "")
                elif 'ref' in rs.attrib:
                    relatedPerson = "ref"
                else:
                    relatedPerson = "None"
            
                personelement = rs.xpath("ancestor::*[@type='person'][1]")
                print("-ParentNode: %s" %personelement[0].tag)
                if personelement is not None:
                    print("--Attribute: %s" %personelement[0].attrib)
                    personref = personelement[0].attrib['ref'].replace("#","")
                else:
                    personref = " "
            except:
                errorarray =[filename, rs.tag, rs.attrib, rs.xpath("normalize-space(.//text())")]
                listoferrors.append(errorarray)
                writeerrorfile(listoferrors)




            texts.append(normed[0])
            norms.append(normed[1])
            persons.append(personref)
            sex.append(getsex(personlistxml, personref))
            relatedPersons.append(relatedPerson)
            if "pe__" in relatedPerson:
                relatedSex.append(getsex(personlistxml, relatedPerson))
            else:
                relatedSex.append(" ")
            regests.append(actualregest)
            if len(date) == 2:
                persdate = date
                regest_dates.append(persdate[0])
                fromdate.append("")
                todate.append("")
                years.append(persdate[1])
            elif len(date) == 3:
                fromtodate = date
                regest_dates.append("")
                fromdate.append(fromtodate[0])
                todate.append(fromtodate[1])
                years.append(fromtodate[2])
    
    
    
    Frame["Person"] = persons
    Frame['sex'] = sex
    Frame[type] = texts
    Frame["normed"] = norms
    Frame["Related Person"] = relatedPersons
    Frame['rel_sex'] = relatedSex
    Frame["xml"] = regests
    Frame["Year"] = years
    Frame["Date"] = regest_dates
    Frame["from"] = fromdate
    Frame["to"] =todate


    Frame.to_csv("../data/output/csv/"+type+"Relations.csv", sep="\t", encoding="utf-8")
    
    writer = pd.ExcelWriter('../data/output/xlsx/'+type+'Relations.xlsx')
    Frame.to_excel(writer, type+'Relations', index=False)
    writer.save()
    





def writeerrorfile(listoferrors):
    with open("../data/output/relations_ERROR.txt", "w") as file:
        if len(listoferrors) > 0:
            file.write(datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
            file.write('\n')
            for error in listoferrors:
                file.write("Error in: %s \n" %error[0] )
                file.write("-Node %s \n" %error[1])
                file.write("-attributes: ")
                for attribute in error[2]:
                    file.write("%s; " %attribute)
                file.write("\n")
                file.write('-Nodetext: "%s"\n' %error[3])
                file.write("-------------\n")
        else:
            file.write(datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S'))
            file.write("\nKeine ERRORS gefunden")


def createdate(node):
    if 'when' in node.attrib:
        when = node.attrib['when'].replace('-','')
        date = datetime.strptime(when, '%Y%m%d').date()
        year = str(date).split("-")[0]
        return [date,year]

    elif 'notAfter' in node.attrib:
        notafter = node.attrib['notAfter']
        if "-" in notafter:
            date = datetime.strptime(notafter, '%Y-%m-%d').date()
        else:
            date = datetime.strptime(notafter, '%Y').date()
        year = str(date).split("-")[0]
        return [date, year]
    elif 'from' in node.attrib:
        fr =node.attrib['from'].replace('-','')
        to = node.attrib['to'].replace('-','')
        fromdate = datetime.strptime(fr, '%Y%m%d').date()
        todate = datetime.strptime(to, '%Y%m%d').date()
        year = str(fromdate).split("-")[0]
        if fromdate == todate:
            return[fromdate,year]
        else:
            return [fromdate, todate,year]
    


def getrolenormlist():
    role_mapping = pd.read_csv("https://docs.google.com/spreadsheets/d/1_ygvqRnDTg6Rx2wp-RNPvZ3cKQFxfDPo3d7SBDiLO5c/export?gid=0&format=csv", dtype=str)\
    .iloc[:,1:3]
    rolesnormlist = role_mapping.set_index("role").to_dict()

    return rolesnormlist['role_norm']


def normrole(rolenormlist, role):
    if role.lower() in rolenormlist:
        return [role, rolenormlist[role.lower()]]
    elif role in rolenormlist:
        return [role, rolenormlist[role]]
    else:
        return [role, " "]


def getsex(personlistxml, id):
    return personlistxml.xpath(".//tei:person[@xml:id='"+id+"']", namespaces=ns)[0].attrib['sex']


normrolelist = getrolenormlist()
personlistxml = readlist("../../indices/lists", "personList.xml")


readfiles(normrolelist, personlistxml, "../../sources/QGW/regests_QGW_1370-1404/done", "off") #Pfad (Durchsuchung), @type of relation


--------------------------
1000.xml
---------
-Node: {http://www.tei-c.org/ns/1.0}roleName
--Attribute: {'type': 'off', 'corresp': 'org__gumpoldskirchen-pfarre'}
--Text: sankcherr
-ParentNode: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': 'pe__niklas_grinzinger_QGW_II_I_1000'}
--------------------------
1001.xml
--------------------------
1002.xml
--------------------------
1003.xml
--------------------------
1004.xml
--------------------------
1005.xml
--------------------------
1006.xml
--------------------------
1007.xml
---------
-Node: {http://www.tei-c.org/ns/1.0}roleName
--Attribute: {'type': 'off', 'corresp': 'org__passau-dioezese'}
--Text: Bischof
-ParentNode: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': 'pe__johann_von_scharfenberg_QGW_II_I_1007'}
---------
-Node: {http://www.tei-c.org/ns/1.0}roleName
--Attribute: {'type': 'off', 'corresp': 'org__passau-dioezese'}
--Text: Bischof
-ParentNode: {http://www.tei-c.org/ns/1.0}rs
-

--------------------------
1028.xml
---------
-Node: {http://www.tei-c.org/ns/1.0}roleName
--Attribute: {'type': 'off', 'corresp': 'org__oesterreich-herzogtum'}
--Text: huebmaister
-ParentNode: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': 'pe__johann_von_tierna_QGW_II_I_1018'}
---------
-Node: {http://www.tei-c.org/ns/1.0}roleName
--Attribute: {'type': 'off', 'corresp': 'org__oesterreich-herzogtum'}
--Text: vorstmaister
-ParentNode: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': 'pe__johann_von_dietreichstokch_QGW_II_I_1028'}
--------------------------
1029.xml
---------
-Node: {http://www.tei-c.org/ns/1.0}roleName
--Attribute: {'type': 'off', 'corresp': 'org__laab-gemeinde'}
--Text: richter
-ParentNode: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': 'pe__ulrich_eibensprunner_QGW_II_I_1029'}
--------------------------
1030.xml
---------
-Node: {http://www.tei-c.org/ns/1.0}roleName
--Attribute: {'type': 'off', 'co

--------------------------
1048.xml
---------
-Node: {http://www.tei-c.org/ns/1.0}roleName
--Attribute: {'type': 'off', 'corresp': 'org__wien-st_niklas_vor_dem_stubentor_zisterzienserinnen'}
--Text: schaffer
-ParentNode: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': 'pe__martin_reisenperger_ii_QGW_II_I_1048'}
--------------------------
1049.xml
---------
-Node: {http://www.tei-c.org/ns/1.0}roleName
--Attribute: {'type': 'off', 'corresp': 'org__oesterreich-herzogtum'}
--Text: hansgraf
-ParentNode: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': 'pe__heinrich_wuerfel_ii_QGW_II_I_1049'}
--------------------------
1050.xml
---------
-Node: {http://www.tei-c.org/ns/1.0}roleName
--Attribute: {'type': 'off', 'corresp': 'org__wien-st_johann_in_der_kaerntnerstrasse'}
--Text: phleger und verbeser
-ParentNode: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': 'pe__johann_QGW_II_I_1050'}
---------
-Node: {http://www.tei-c.org/ns/

---------
-Node: {http://www.tei-c.org/ns/1.0}roleName
--Attribute: {'type': 'off', 'corresp': 'org__arbesthal-pfarre'}
--Text: pharrer
-ParentNode: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': 'pe__ulrich_von_fischamend_QGW_II_I_1072'}
---------
-Node: {http://www.tei-c.org/ns/1.0}roleName
--Attribute: {'type': 'off', 'corresp': 'org__hainburg-burgkapelle'}
--Text: Kaplan
-ParentNode: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': 'pe__ulrich_von_fischamend_QGW_II_I_1072'}
--------------------------
1073.xml
---------
-Node: {http://www.tei-c.org/ns/1.0}roleName
--Attribute: {'type': 'off', 'corresp': 'org__wien'}
--Text: purgermaister
-ParentNode: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': 'pe__paul_holzkaeufel_QGW_II_I_1010'}
---------
-Node: {http://www.tei-c.org/ns/1.0}roleName
--Attribute: {'type': 'off', 'corresp': 'org__wien'}
--Text: gesworner des rats
-ParentNode: {http://www.tei-c.org/ns/1.0}rs
--A

---------
-Node: {http://www.tei-c.org/ns/1.0}roleName
--Attribute: {'type': 'off', 'corresp': 'org__passau-dioezese'}
--Text: commissarius officialatus curie Pataviensis
-ParentNode: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': 'pe__peter_QGW_II_I_1093bis'}
--------------------------
1094.xml
---------
-Node: {http://www.tei-c.org/ns/1.0}roleName
--Attribute: {'type': 'off', 'corresp': 'org__klosterneuburg-augustiner_chorherren'}
--Text: amptman
-ParentNode: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': 'pe__ulrich_tumbriczer_QGW_II_I_1082'}
---------
-Node: {http://www.tei-c.org/ns/1.0}roleName
--Attribute: {'type': 'off', 'corresp': 'org__wien'}
--Text: rates der statt ze Wienn
-ParentNode: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': 'pe__michael_vinkch_QGW_II_I_1082'}
---------
-Node: {http://www.tei-c.org/ns/1.0}roleName
--Attribute: {'type': 'off', 'corresp': 'org__wien'}
--Text: Ratsherr
-ParentNode: {

...

---------
-Node: {http://www.tei-c.org/ns/1.0}roleName
--Attribute: {'type': 'off', 'corresp': 'org__wien'}
--Text: rats der stat ze Wienne
-ParentNode: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': 'pe__pilgrim_snaeczl_QGW_II_I_1015'}
--------------------------
900.xml
---------
-Node: {http://www.tei-c.org/ns/1.0}roleName
--Attribute: {'type': 'off', 'corresp': 'org__wien-st_maria_im_rathaus'}
--Text: caplan
-ParentNode: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': 'pe__jakob_poll_QGW_II_I_770'}
--------------------------
901.xml
---------
-Node: {http://www.tei-c.org/ns/1.0}roleName
--Attribute: {'type': 'off', 'corresp': 'org__passau-dioezese'}
--Text: Bischof
-ParentNode: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': 'pe__albrecht_bischof_von_passau_QGW_II_I_1007'}
--------------------------
902.xml
---------
-Node: {http://www.tei-c.org/ns/1.0}roleName
--Attribute: {'type': 'off', 'corresp': 'org__passau-

---------
-Node: {http://www.tei-c.org/ns/1.0}roleName
--Attribute: {'type': 'off', 'corresp': 'org__klosterneuburg-augustiner_chorherren'}
--Text: chorherr und obrister chelner
-ParentNode: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': 'pe__peter_lenhofer_QGW_II_I_1001'}
---------
-Node: {http://www.tei-c.org/ns/1.0}roleName
--Attribute: {'type': 'off', 'corresp': 'org__wien-st_philipp_und_st_jakob_kapelle-altar_st_maria-messe_leonhard_poll_QGW_II_I_889_a'}
--Text: Kaplan
-ParentNode: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': 'pe__weichart_von_zistersdorf_QGW_II_I_798'}
--------------------------
917.xml
---------
-Node: {http://www.tei-c.org/ns/1.0}roleName
--Attribute: {'type': 'off', 'corresp': 'org__mahrenberg-kloster'}
--Text: priorinn
-ParentNode: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': 'pe__vromut_QGW_II_I_917'}
--------------------------
918.xml
---------
-Node: {http://www.tei-c.org/ns/1.0}ro

--Attribute: {'type': 'off', 'corresp': 'org__moedling-gemeinde'}
--Text: judenrichter
-ParentNode: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': 'pe__michael_cholb_QGW_II_I_936'}
--------------------------
937.xml
---------
-Node: {http://www.tei-c.org/ns/1.0}roleName
--Attribute: {'type': 'off', 'corresp': 'org__melk-benediktiner'}
--Text: abt
-ParentNode: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': 'pe__friedrich_QGW_II_I_872'}
---------
-Node: {http://www.tei-c.org/ns/1.0}roleName
--Attribute: {'type': 'off', 'corresp': 'org__enns'}
--Text: haubtman
-ParentNode: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': 'pe__heinrich_von_wallsee_ii_QGW_II_I_1344'}
--------------------------
938.xml
--------------------------
939.xml
---------
-Node: {http://www.tei-c.org/ns/1.0}roleName
--Attribute: {'type': 'off', 'corresp': 'org__wien-st_stephan-kapelle_st_erasmus_im_neuen_karner_auf_dem_st_stephansfriedhof-zeche_sch

--------------------------
954.xml
---------
-Node: {http://www.tei-c.org/ns/1.0}roleName
--Attribute: {'type': 'off', 'corresp': 'org__passau-dioezese'}
--Text: Bischof
-ParentNode: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': 'pe__albrecht_bischof_von_passau_QGW_II_I_1007'}
--------------------------
955.xml
---------
-Node: {http://www.tei-c.org/ns/1.0}roleName
--Attribute: {'type': 'off', 'corresp': 'org__oesterreich-herzogtum'}
--Text: hubmaister
-ParentNode: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': 'pe__friedrich_von_tierna_ii_QGW_II_I_1106'}
--------------------------
956.xml
---------
-Node: {http://www.tei-c.org/ns/1.0}roleName
--Attribute: {'type': 'off', 'corresp': 'org__wien'}
--Text: judenrichter
-ParentNode: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': 'pe__niklas_magseit_QGW_II_I_779'}
--------------------------
957.xml
---------
-Node: {http://www.tei-c.org/ns/1.0}roleName
--Attribute: {'t

-ParentNode: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': 'pe__jakob_poll_QGW_II_I_770'}
---------
-Node: {http://www.tei-c.org/ns/1.0}roleName
--Attribute: {'type': 'off', 'corresp': 'org__oesterreich-herzogtum'}
--Text: huebmaister
-ParentNode: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': 'pe__johann_von_tierna_QGW_II_I_1018'}
--------------------------
986.xml
---------
-Node: {http://www.tei-c.org/ns/1.0}roleName
--Attribute: {'type': 'off', 'corresp': 'org__wien-st_peter-altar_st_katharina'}
--Text: Kaplan
-ParentNode: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': 'pe__ulrich_von_kirchberg_QGW_II_I_986'}
--------------------------
987.xml
---------
-Node: {http://www.tei-c.org/ns/1.0}roleName
--Attribute: {'type': 'off', 'corresp': 'org__wien'}
--Text: rates der stat zu/e Wienne
-ParentNode: {http://www.tei-c.org/ns/1.0}rs
--Attribute: {'type': 'person', 'ref': 'pe__michael_geukramer_ii_QGW_II_I_1235'}
--